## Insert PatientDatasets into Database
- [ ] exp.json => Exps
- [ ] info.json => Patients
- [ ] normal.csv => Inspections

In [8]:
!ls

AUTHORS               app                   supervisor-app.conf
Dockerfile            app.sock              uwsgi.ini
Dockerfile-py2        insert_datasets.ipynb uwsgi_params
PatientsDataset       nginx-app.conf
README.md             requirements.txt


In [16]:
import pandas as pd
import json
import glob
import os

In [11]:
head_dir = "./PatientsDataset"
query = "patient*"
patient_dir_list = glob.glob(os.path.join(head_dir,query))

In [ ]:
normal_file = "normal.csv"
info_file = "info.json"
exp_html_file = "exp.html"
exp_json_file = "exp.json"

In [33]:
Patient.objects.update_or_create?

In [46]:
def InsertPatient(info_json, patient_id):
    info_json["patient_text"] = "patient " + str(info_json["patient_id"])
    del info_json["patient_id"]
    result, created = Patient.objects.update_or_create(
        id=patient_id,
        defaults=info_json,
    )

def InsertExp(exp_json, patient_id):
    """
    ExpをDatabaseにInsert
    arg:
    exp_json ( = '{"ALB t_0 < 3.3": 0.1,"BUN": 0.05}')
    """
    for k, v in exp_json.items():
        Exp.objects.update_or_create(
            patient=Patient(patient_id),
            cond_exp=k,
            defaults={"contribution":v},
        )
        print(patient_id, k, v)
        
        
def InsertInspection():
    pass

def insert_dataset():
    head_dir = "./PatientsDataset"
    query = "patient*"
    normal_file = "normal.csv"
    info_file = "info.json"
    exp_html_file = "exp.html"
    exp_json_file = "exp.json"
    patient_dir_list = glob.glob(os.path.join(head_dir,query))
    for patient_dir in patient_dir_list:
        patient_id = int(os.path.basename(patient_dir)[7:])
        normal_path = os.path.join(patient_dir, normal_file)
        info_path = os.path.join(patient_dir, info_file)
        exp_html_path = os.path.join(patient_dir, exp_html_file)
        exp_json_path = os.path.join(patient_dir, exp_json_file)
        patient_df = pd.read_csv(normal_path)
        with open(info_path, "r") as f:
            info_json = json.load(f)
        with open(exp_json_path, "r") as f:
            exp_json = json.load(f)
        InsertPatient(info_json, patient_id)
        InsertExp(exp_json, patient_id)
        
        

In [47]:
insert_dataset()

10000 IP 0.4398872543682142
10000 ALB 0.4125634410619853
10000 CRP 0.1090309828872913
10000 K 0.7547324065891593
10000 PCR 0.5101465845342399
10001 CRE 0.7585278706905739
10001 Ca 0.32925632114148495
10001 BUN 0.29252981670089173
10001 ALB 0.8310761883095381
10001 IP 0.08927017027511408
10002 Cl 0.9484336194396933
10002 T-CHO 0.46647162632328554
10002 IP 0.2649341432352903
10002 ALB 0.6658763417886838
10002 BUN 0.2491750178827411
10003 Ca 0.789590993924747
10003 T-CHO 0.8407725766984647
10003 Cl 0.814395199090939
10003 Na 0.7205176565763842
10003 CRE 0.25430831190910197
10004 Ca 0.6240856634538372
10004 BUN 0.20525470834248216
10004 Cl 0.7287003425019891
10004 CRE 0.8151471949919927
10004 CRP 0.3527232266282685
10005 Na 0.6053085539079532
10005 ALB 0.2730371086233455
10005 BUN 0.9866693965206156
10005 PCR 0.4039469874647169
10005 Cl 0.3239240595748236
10006 BUN 0.384184981002594
10006 CRP 0.37287993906287464
10006 Cl 0.4696785557714691
10006 TP 0.7036933983425584
10006 IP 0.56399714450

In [26]:
os.path.exists("./PatientsDataset")

True

In [27]:
!ls PatientsDataset/

patient10000 patient10002 patient10004 patient10006 patient10008
patient10001 patient10003 patient10005 patient10007 patient10009


In [12]:
test = './PatientsDataset/patient1'

In [15]:
os.path.basename(test)[7:]

'1'

In [14]:
import re